In [1]:
%load_ext autoreload
%autoreload 2
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, material = read_VSP_group(pappas_only=False)

D = count_matrix(groups_VSP, raw_feature.shape[0])
clusters_VSP = spectralClusteringDBSCAN(D, threshold=0,eps=0.001)

print(raw_feature.shape)
print(label_material.shape)

INFO:visiprog.data:After removing non zero entries, N is 3708
INFO:visiprog.data:Number of disconnected elements is 161

(5245, 82)
(5245,)


In [4]:
# print(clusters_VSP)

In [5]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])
label_VSP_clusters = get_VSP_label(clusters_VSP, raw_feature.shape[0])

In [7]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)

In [33]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

list_img = read_img_list()

n_cluster = 2

for i in range(10):
    X_train, X_test, X_STSIMM_train, X_STSIMM_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, X_STSIMM, label_material, label_visiprog, \
                     test_size=0.2, stratify=label_material)
    
    res = kmean_subclass(X_STSIMM_train, Y_mat_train, X_STSIMM_test, Y_mat_test, n_cluster)
    accuracy_STSIMM.append(res['accuracy'])
    
    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_train = model.transform(X_train)
        X_mat_test = model.transform(X_test)
        res = kmean_subclass(X_mat_train, Y_mat_train, X_mat_test, Y_mat_test, n_cluster)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        acuracy_mat.append(np.nan)
    
    model_vsp_original = LFDA()
    try:

        idx_outlier = np.where(Y_VSP_train == 0)[0]
        X_train_without_outlier = X_train[~idx_outlier,:]
        Y_VSP_train_without_outlier = Y_VSP_train[~idx_outlier]
        model_vsp_original.fit(X_train_without_outlier, Y_VSP_train_without_outlier)
        
        X_vsp_original_train = model_vsp_original.transform(X_train)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = kmean_subclass(X_vsp_original_train, Y_mat_train, \
                                          X_vsp_original_test, Y_mat_test, n_cluster)
        print(res_vsp_original['accuracy'])
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        print(e)
        accuracy_vsp_original.append(np.nan)


    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_train = model_vsp.transform(X_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = kmean_subclass(X_vsp_train, Y_mat_train, X_vsp_test, Y_mat_test, n_cluster)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(np.nan)
        

df = pd.DataFrame({'STSIM-M':accuracy_STSIMM,'visiprog':accuracy_vsp, \
                   'visiprog_original': accuracy_vsp_original, 'material':accuracy_mat}, \
                  columns=['STSIM-M','visiprog_original','visiprog','material'])
df.loc['average'] = df.mean()
display(df)


df.to_csv('thrasos/within_trainedongroups_k{}.csv'.format(n_cluster))



INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[28 58 38 ..., 21 34 18]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[28 58 38 ..., 21 34 18]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[28 58 38 ..., 21 34 18]
INFO:visiprog.evaluate:Running on test set
0.858913250715
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[28 58 38 ..., 21 34 18]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[ 9 47 40 ..., 19 43 58]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[ 9 47 40 ..., 19 43 58]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[ 9 47 40 ..., 19 43 58]
INFO:visiprog.evaluate:Running on test set
0.879885605338
INFO:visiprog.evaluate:KMeans clustering
INFO:

,STSIM-M,visiprog_original,visiprog,material
0,0.851287,0.858913,0.883699,0.918970
1,0.867493,0.879886,0.878932,0.929457
2,0.846520,0.843661,0.877979,0.936130
3,0.872259,0.909438,0.900858,0.953289
4,0.859867,0.846520,0.877026,0.936130
5,0.851287,0.874166,0.857007,0.910391
6,0.864633,0.846520,0.876072,0.910391
7,0.859867,0.854147,0.903718,0.927550
8,0.855100,0.830315,0.857960,0.933270
9,0.859867,0.860820,0.874166,0.914204
